In [ ]:

import cv2
import mediapipe as mp
import numpy as np
import pyttsx3

# Text-to-speech
engine = pyttsx3.init()
def speak(text):
    engine.say(text)
    engine.runAndWait()

# Mediapipe face mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)

# נקודות עין
LEFT_IRIS = [474, 475, 476, 477]

# הגדרות
SCREEN_WIDTH, SCREEN_HEIGHT = 1280, 720
CELL_ROWS, CELL_COLS = 4, 4
CELL_WIDTH = SCREEN_WIDTH // 2 // CELL_COLS
CELL_HEIGHT = SCREEN_HEIGHT // CELL_ROWS

# טקסטים לכל תא
menu_texts = [
    ["רעבה", "עייפה", "כואבת לי הבטן", "קור"],
    ["כואב לי הראש", "רוצה הביתה", "שמח/ה", "עצוב/ה"],
    ["רוצה לשחק", "לבד", "חיבוק", "מים"],
    ["כן", "לא", "שירותים", "יציאה"]
]

# אתחול מצלמה
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

last_direction = ""
cooldown = 30
counter = 0

def midpoint(p1, p2):
    return ((p1[0] + p2[0]) // 2, (p1[1] + p2[1]) // 2)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    frame_resized = cv2.resize(frame, (SCREEN_WIDTH // 2, SCREEN_HEIGHT))
    rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb)

    # תפריט ריק
    menu_area = np.full((SCREEN_HEIGHT, SCREEN_WIDTH // 2, 3), 255, dtype=np.uint8)

    # ציור תפריט 4x4
    for row in range(CELL_ROWS):
        for col in range(CELL_COLS):
            x1 = col * CELL_WIDTH
            y1 = row * CELL_HEIGHT
            x2 = x1 + CELL_WIDTH
            y2 = y1 + CELL_HEIGHT
            cv2.rectangle(menu_area, (x1, y1), (x2, y2), (200, 230, 255), -1)
            cv2.rectangle(menu_area, (x1, y1), (x2, y2), (100, 100, 100), 2)
            text = menu_texts[row][col]
            cv2.putText(menu_area, text, (x1 + 10, y1 + 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)

    # חיבור תפריט עם תמונת מצלמה
    combined = np.hstack((menu_area, frame_resized))

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            iris = [face_landmarks.landmark[i] for i in LEFT_IRIS]
            center = midpoint(
                (int(iris[0].x * (SCREEN_WIDTH // 2)), int(iris[0].y * SCREEN_HEIGHT)),
                (int(iris[2].x * (SCREEN_WIDTH // 2)), int(iris[2].y * SCREEN_HEIGHT))
            )
            center_global = (center[0] + SCREEN_WIDTH // 2, center[1])  # מסך מלא
            cv2.circle(combined, center_global, 5, (0, 0, 255), -1)

            direction = ""
            if center[0] < SCREEN_WIDTH // 2:
                col = center[0] // CELL_WIDTH
                row = center[1] // CELL_HEIGHT
                if 0 <= row < CELL_ROWS and 0 <= col < CELL_COLS:
                    direction = menu_texts[int(row)][int(col)]
                    cv2.rectangle(combined, (int(col)*CELL_WIDTH, int(row)*CELL_HEIGHT),
                                  ((int(col)+1)*CELL_WIDTH, (int(row)+1)*CELL_HEIGHT), (0, 255, 0), 3)
                    cv2.putText(combined, f"מבט: {direction}", (30, 40),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            if direction and direction != last_direction and counter == 0:
                speak(direction)
                last_direction = direction
                counter = cooldown

                if direction == "יציאה":
                    cap.release()
                    cv2.destroyAllWindows()
                    break

    if counter > 0:
        counter -= 1

    cv2.imshow("Eye Menu 4x4", combined)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()
